<center>

# Steam Games ETL

<center>

In [26]:
# Importations.
import pandas as pd
import ast
import dateutil.parser
import numpy as np
import re
import os
import json

<div style="text-align: justify">

### 1. Converting Data: From JSON Format to CSV Format

This is the final version of the code to transform the JSON file into a CSV file. I updated the code to drop all the Nan rows and reset the index to work without compatibily issues.

</div>

In [78]:
# File paths.
steam_games = 'PI MLOps - STEAM/steam_games.json'
steam_games_csv = 'csv/steam_games.csv'

# If the file does not exist, create the file.
if not os.path.exists(steam_games_csv):
    # reading the Json data file.
    games = []
    with open(steam_games, 'r') as f:
        # Iterates over the lines in the file and loads each line as a JSON object.
        for line in f:
            json_object = json.loads(line)
            games.append(json_object)

    # transform the Json file into a dataframe.
    df_games = pd.DataFrame(games)

    # Apply dropna and reset_index
    df_games = df_games.dropna(how='all').reset_index(drop=True)

    # save the dataframe as a CSV file in the csv folder.
    df_games.to_csv(steam_games_csv, index=False)
    print(f'The file {steam_games_csv} was successfully created.')
else:
    print(f'The file {steam_games_csv} already exists.')

The file csv/steam_games.csv was successfully created.


In [2]:
# Reading the csv file.
df_steam_games = pd.read_csv('csv/steam_games.csv')

C:\Users\Alvaro Javier\AppData\Local\Temp\ipykernel_1436\133674220.py:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df_steam_games = pd.read_csv('csv/steam_games.csv')


When I read the file, I got a warning message (DtypeWarning) saying that the columns had mixed types. Yet, I decided to saw it.

In [3]:
df_steam_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"['Strategy', 'Indie', 'Casual', 'Simulation']",http://steamcommunity.com/app/773640/reviews/?...,"['Single-player', 'Steam Achievements']",1.99,False,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"['Strategy', 'Indie', 'Casual']",http://steamcommunity.com/app/733530/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,733530.0,Sacada
120442,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"['Indie', 'Simulation', 'Racing']",http://steamcommunity.com/app/610660/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",1.99,False,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"['Casual', 'Indie']",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"['Indie', 'Casual', 'Puzzle', 'Singleplayer', ...",http://steamcommunity.com/app/658870/reviews/?...,"['Single-player', 'Steam Achievements', 'Steam...",4.99,False,658870.0,"xropi,stev3ns"


The file is full of NaN data, and I also noticed that the genre column is nested. First, I wanted to remove all the NaN values and reset the index to have a better look at the file.

In [79]:
df_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com


This is the result after the code update.

In [80]:
df_games.shape

(32135, 13)

In [81]:
# Define a function to safely apply ast.literal_eval
def safe_literal_eval(x):
    try:
        return ast.literal_eval(x)
    except (ValueError, SyntaxError):
        return x

# Convert string representation of lists to actual lists, handling NaN values
df_games['genres'] = df_games['genres'].apply(safe_literal_eval)

# Apply explode to the 'genres' column.
df_games = df_games.explode('genres')
df_games = df_games.dropna(subset=['genres'])

As the column genres was important for the project I removed all the Nan values.

In [82]:
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
0,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
0,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
0,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
0,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32132,Laush Studio,Indie,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
32132,Laush Studio,Racing,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
32132,Laush Studio,Simulation,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
32133,SIXNAILS,Casual,EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"


<div style="text-align: justify">

This is the result after I unnested the column genres, the tag column is similar but this column contains the genre and key words of the game.

</div>

<div style="text-align: justify">

### 2. Finding null and duplicate values.

After I deleted all Nan values at the begining of the file and by the genres column during the unnesting process, I wanted to know if the other key columns had Nan values and if was possible to complete or clean that information.

</div>

In [83]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71554 entries, 0 to 32133
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     59115 non-null  object
 1   genres        71554 non-null  object
 2   app_name      71552 non-null  object
 3   title         71552 non-null  object
 4   url           71554 non-null  object
 5   release_date  71501 non-null  object
 6   tags          71508 non-null  object
 7   reviews_url   71552 non-null  object
 8   specs         70997 non-null  object
 9   price         68355 non-null  object
 10  early_access  71554 non-null  object
 11  id            71552 non-null  object
 12  developer     71204 non-null  object
dtypes: object(13)
memory usage: 7.6+ MB


In [84]:
# Get the count of null values per column
null_counts = df_games.isnull().sum()

# Display the information
null_counts

publisher       12439
genres              0
app_name            2
title               2
url                 0
release_date       53
tags               46
reviews_url         2
specs             557
price            3199
early_access        0
id                  2
developer         350
dtype: int64

I checked if there are null values, and after the unnesting process, I checked that the columns(title, id, developer,release date) I considered important for the project still presented null data.

In [85]:
# Finding duplicates by both 'id' and 'genres'
duplicates = df_games[df_games.duplicated(subset=['id', 'genres'], keep=False)]

# Print the DataFrame containing duplicates by 'id' and 'genres'
duplicates

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
13894,Bethesda Softworks,Action,Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games
14573,Bethesda Softworks,Action,Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,False,612880,Machine Games


In [86]:
# deleting duplicate values.
# Get the total number of rows before deduplication.
total_rows_before = len(df_games)

# Remove duplicate rows based on 'id' and 'genres', keeping the last occurrences
df_games = df_games.drop_duplicates(subset=['id', 'genres'], keep='first')

# Get the total number of rows after deduplication.
total_rows_after = len(df_games)

# Calculate the number of rows removed.
rows_removed = total_rows_before - total_rows_after

# Print the information.
print(f'Total rows before: {total_rows_before}')
print(f'Total rows after: {total_rows_after}')
print(f'Rows removed: {rows_removed}')

Total rows before: 71554
Total rows after: 71553
Rows removed: 1


The file only had a duplicate item, after check both lines had the same information I removed the duplicated.

<div style="text-align: justify">

### 3. Checking the **release date** column and creating the **year** column.

Realease date was an important column because this contained the years that I used to answer the queries.

</div>

In [87]:
df_games = df_games.dropna(subset=['release_date'])

I started deleting the null results, because was not possible to find a way to complete the information.

In [88]:
# Get unique values from the 'release_date' column
df_unique = df_games['release_date'].unique()

# Desired number of columns
num_cols = 7

# Set the maximum column width
max_column_width = 10

# Calculate the number of necessary rows
num_rows = -(-len(df_unique) // num_cols)

# Print unique values in the specified number of columns
for i in range(num_rows):
    row_values = df_unique[i*num_cols:(i+1)*num_cols]
    formatted_row = [str(value)[:max_column_width].ljust(max_column_width) for value in row_values]
    print('\t'.join(formatted_row))

2018-01-04	2017-07-24	2017-12-07	Soon..    	2018-01-03	2017-12-22	2017-12-23
1997-06-30	1998-11-08	2016-11-25	2018-01-01	2017-12-30	2006-07-06	2006-07-11
2017      	Beta测试已开启 	2017-12-29	2018-03-30	2005-08-09	2006-09-29	2006-11-20
2006-11-29	2006-11-24	2006-12-14	2006-12-19	2003-08-23	2006-12-21	2006-04-17
2006-08-01	2005-07-12	2007-06-26	2006-07-24	2002-11-12	2000-11-17	2003-10-23
2006-10-16	1998-10-31	2007-06-01	1995-04-01	2007-06-05	2006-05-23	2006-10-17
1996-06-17	2007-06-29	2006-12-20	1995-04-30	1995-06-01	1994-05-05	1994-08-03
2001-11-20	1997-02-28	1993-10-10	2003-09-09	2007-08-03	2007-05-31	2007-03-20
2006-01-01	2007-08-21	2006-10-09	2004-09-22	2006-06-26	2007-12-14	1998-06-30
1999-10-25	2004-04-20	2003-07-01	2003-10-14	2006-04-07	2001-07-25	2008-10-28
2007-10-10	2007-10-23	2007-08-01	2007-09-01	2007-12-03	2008-02-08	2007-05-08
2006-02-21	2006-05-02	2007-10-16	2006-05-30	2006-10-26	2005-03-14	2003-02-03
1998-04-30	2008-03-21	2003-02-18	2004-03-23	2000-10-25	2006-12-12	2005-03-15

I found the column release date had diferente date formats, like 1997-06-30 - January 20 - 2016 - Q1, 2018 - 15.01.2018 - Soon.. - Jun 2009 - 2018 early.

In [89]:
# Define the extract_year function
def extract_year(date_str):
    try:
        # Try to parse the date using dateutil.parser
        parsed_date = dateutil.parser.parse(str(date_str))

        # Extract the year from the parsed date
        return parsed_date.year
    except Exception as e:
        # Handle format errors or invalid dates
        print(f"Error parsing date: {date_str}. Error: {str(e)}")
        return 'Incorrect format'

# Apply the extract_year function to the 'release_date' column and create the 'year' column
df_games['year'] = df_games['release_date'].apply(extract_year)

# Show the DataFrame with the new 'year' column
print(df_games[['release_date', 'year']])

Error parsing date: Soon... Error: Unknown string format: Soon..
Error parsing date: Soon... Error: Unknown string format: Soon..
Error parsing date: Soon... Error: Unknown string format: Soon..
Error parsing date: Soon... Error: Unknown string format: Soon..
Error parsing date: Beta测试已开启. Error: Unknown string format: Beta测试已开启
Error parsing date: Beta测试已开启. Error: Unknown string format: Beta测试已开启
Error parsing date: Beta测试已开启. Error: Unknown string format: Beta测试已开启
Error parsing date: Beta测试已开启. Error: Unknown string format: Beta测试已开启
Error parsing date: Coming Soon. Error: Unknown string format: Coming Soon
Error parsing date: Coming Soon. Error: Unknown string format: Coming Soon
Error parsing date: Q2 2017. Error: Unknown string format: Q2 2017
Error parsing date: Q2 2017. Error: Unknown string format: Q2 2017
Error parsing date: Q2 2017. Error: Unknown string format: Q2 2017
Error parsing date: Coming Soon. Error: Unknown string format: Coming Soon
Error parsing date: Coming Soo

All the formats that was not possible to get the date were transformed into incorrect format.

In [90]:
df_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,year
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018


I verified that the code worked.

In [91]:
# Get unique values from the 'year' column
df_year = df_games['year'].unique()

# Desired number of columns
num_cols = 7

# Set the maximum column width
max_column_width = 10

# Calculate the number of necessary rows
num_rows = -(-len(df_year) // num_cols)

# Print unique values in the specified number of columns
for i in range(num_rows):
    row_values = df_year[i*num_cols:(i+1)*num_cols]
    formatted_row = [str(value)[:max_column_width].ljust(max_column_width) for value in row_values]
    print('\t'.join(formatted_row))

2018      	2017      	Incorrect 	1997      	1998      	2016      	2006      
2005      	2003      	2007      	2002      	2000      	1995      	1996      
1994      	2001      	1993      	2004      	1999      	2008      	2009      
1992      	1989      	2010      	2011      	2013      	2012      	2014      
1983      	1984      	2015      	1990      	1988      	1991      	1987      
2024      	1986      	2021      	2019      	1985      


I checked the data values of the year column.

In [93]:
# Count how many "Incorrect format" values are in the 'year' column
incorrect_format_count = (df_games['year'] == 'Incorrect format').sum()

# Print the result
print(f"Number of 'Incorrect format': {incorrect_format_count}")

Number of 'Incorrect format': 489


In [94]:
# Eliminate all rows where 'year' is 'Incorrect format'
df_games = df_games[df_games['year'] != 'Incorrect format']

In [95]:
# Rename the 'id' column to 'item_id' and apply the changes inplace
df_games.rename(columns={'id': 'item_id'}, inplace=True)

In [96]:
# Function to extract the code from the URL
def extract_app_code(url):
    if pd.isna(url):
        return None
    match = re.search(r'/app/(\d+)/?', url)
    return int(match.group(1)) if match else None

# Function to apply the update logic to the 'item_id' column
def update_item_id(row):
    if pd.isna(row['item_id']):
        app_code = extract_app_code(row['url'])
        return app_code if app_code else np.nan
    else:
        return row['item_id']

# Apply the function to the 'item_id' column
df_games['item_id'] = df_games.apply(update_item_id, axis=1)

In [97]:
# Convert the 'item_id' column to integer format
df_games['item_id'] = df_games['item_id'].astype(int)

The information inside the column item_id was floating so I changed as decimal to avoid problems when I want to conect the tables.

In [98]:
df_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,year
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,2018


In [99]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71011 entries, 0 to 32133
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher     58588 non-null  object
 1   genres        71011 non-null  object
 2   app_name      71009 non-null  object
 3   title         71009 non-null  object
 4   url           71011 non-null  object
 5   release_date  71011 non-null  object
 6   tags          70965 non-null  object
 7   reviews_url   71009 non-null  object
 8   specs         70454 non-null  object
 9   price         68338 non-null  object
 10  early_access  71011 non-null  object
 11  item_id       71011 non-null  int32 
 12  developer     70661 non-null  object
 13  year          71011 non-null  object
dtypes: int32(1), object(13)
memory usage: 7.9+ MB


I checked that the app_name and title had missing data.

In [100]:
# Fill null cells in the columns 'genres', 'app_name', and 'developer' with 'missing data'
df_games['title'] = df_games['title'].fillna('missing data')
df_games['app_name'] = df_games['app_name'].fillna('missing data')
df_games['developer'] = df_games['developer'].fillna('missing data')

In [101]:
# Checking the rows with 'missing data' in the 'app_name' column.
missing_data_rows = df_games[df_games['app_name'] == 'missing data']
missing_data_rows

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,item_id,developer,year
2580,NaN,Action,missing data,missing data,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,False,317160,missing data,2014
2580,NaN,Indie,missing data,missing data,http://store.steampowered.com/app/317160/_/,2014-08-26,"[Action, Indie]",http://steamcommunity.com/app/317160/reviews/?...,"[Single-player, Game demo]",NaN,False,317160,missing data,2014


In [102]:
# Removing rows where 'app_name' is equal to 'missing data'.
df_games = df_games[df_games['app_name'] != 'missing data']

In [103]:
# List of columns to be deleted.
columns_to_drop = ['publisher', 'app_name', 'url', 'release_date', 'tags', 'reviews_url', 'specs', 'price', 'early_access']

# Deleting the columns.
df_games = df_games.drop(columns=columns_to_drop, axis=1)

In [108]:
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71009 entries, 0 to 32133
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   genres     71009 non-null  object
 1   title      71009 non-null  object
 2   item_id    71009 non-null  int32 
 3   developer  71009 non-null  object
 4   year       71009 non-null  object
dtypes: int32(1), object(4)
memory usage: 3.0+ MB


In [107]:
df_games

,genres,title,item_id,developer,year
0,Action,Lost Summoner Kitty,761140,Kotoshiro,2018
0,Casual,Lost Summoner Kitty,761140,Kotoshiro,2018
0,Indie,Lost Summoner Kitty,761140,Kotoshiro,2018
0,Simulation,Lost Summoner Kitty,761140,Kotoshiro,2018
0,Strategy,Lost Summoner Kitty,761140,Kotoshiro,2018
...,...,...,...,...,...
32132,Indie,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
32132,Racing,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
32132,Simulation,Russian Roads,610660,Laush Dmitriy Sergeevich,2018
32133,Casual,EXIT 2 - Directions,658870,"xropi,stev3ns",2017


In [109]:
# Overwrite the original CSV file.
df_games.to_csv('csv/steam_games.csv', index=False)